# Relabel TMA cores

This notebook automatically labels the TMA cores from a JSON file with a standardized naming convention (R1C1, R1C2, etc). This ensures that on a TMA with many different FOVs across many different cores, no mistakes were made in naming. 

The inputs are manually generated JSON file with the selected cores, as well as a separate JSON defining the upper left hand corner and bottom right hand corner of the TMA grid. 

In [ ]:
import sys
sys.path.append('../')

In [ ]:
import json
import os
from skimage.io import imread

from toffy import tiling_utils

# suppress mpl deprecation
import warnings
from matplotlib.cbook import mplDeprecation
warnings.filterwarnings("ignore", category=mplDeprecation)

# 1. Generate the FOVs from a TMA spec file

You will first need to define the prefix to use for your input and output file names. This makes it easy to associate all of the files from a single run with one another. The default is `example_tma`.

The following input files are required for the script to run:

* `tma_corners_path`: the path to the JSON file containing FOVs defining the four corners (top-left, top-right, bottom-left, and bottom-right, in that order). It should be named `example_tma_corners.json`, where `example_tma` is replaced with the `tma_prefix`. 
* `manual_run_path`: the path to your run file with the manually selected FOVs. It should be named `example_tma_manual_run.json`.
* `slide_path`: the path to the slide image of the TMA

In [ ]:
# these variables need to be defined by the user
base_dir = "data_dir"
json_tiling_dir = os.path.join(base_dir, "json_tiling_data")

# define the prefix of each file
tma_prefix = 'example_tma'

# update with path to current slide image
slide_path = os.path.join(path_to_optical_image, 'tma_prefix')

In [ ]:
# these variables should be automatically defined; make sure they're named as described in the comment above 
tma_corners_path = os.path.join(json_tiling_dir, '%s_tma_corners.json' % tma_prefix)
auto_fov_names_path = os.path.join(json_tiling_dir, '%s_automatic_fov_names.json' % tma_prefix)
manual_run_path = os.path.join(json_tiling_dir, '%s_manual_run.json' % tma_prefix)
mapping_path = os.path.join(json_tiling_dir, '%s_mapping.json' % tma_prefix)
remapped_fov_path = os.path.join(json_tiling_dir, '%s_automatic_run.json' % tma_prefix)
moly_path = os.path.join(json_tiling_dir, '%s_moly_point.json' % tma_prefix)

In [ ]:
# Define TMA grid dimensions
tma_num_row = 7
tma_num_col = 4

# generate automatically named TMA
auto_fov_regions = tiling_utils.generate_tma_fov_list(
    tma_corners_path,
    tma_num_row,
    tma_num_col
)

# save the automatically-named TMA FOVs to centroids mapping
with open(auto_fov_names_path, 'w', encoding='utf-8') as afrp:
    json.dump(auto_fov_regions, afrp)

# 2. Define the FOV locations and map to TMA-spec file FOV names

### Load the user-defined FOVs and slide image

In [ ]:
# load the user-defined set of FOVs in
with open(manual_run_path, 'r', encoding='utf-8') as mfop:
    manual_fov_regions = json.load(mfop)

In [ ]:
# load the slide image in
slide_data = imread(slide_path)

### Visualize and remap tiles

Set these parameters to view validation warnings while remapping (mappings are between FOVs in `manual_fov_regions` to FOVs in `auto_fov_regions`):

* `check_dist`: set to a positive value to notify of FOV mappings at a distance greater than this value (measured in microns), sorted by decreasing distance. Set to `None` to bypass.
* `check_duplicates`: set to `True` to notify of FOVs in `auto_fov_regions` with multiple FOVs mapping to it. Set to `False` to bypass.
* `check_mismatches`: set to `True` to notify of FOV mappings with mismatched names. Set to `False` to bypass.

In [ ]:
# define which validation checks to display in the FOV pair validation checks box
check_dist = 2000
check_duplicates = True
check_mismatches = True

Each FOV in `manual_fov_regions` are mapped to their closest corresponding FOV in `auto_fov_regions` by default. To see the current mappings, select FOVs in the `Manually-defined FOV` menu. To remap a manual FOV to a different auto FOV, use the `Automatically-generated FOV` menu.

After you're done finished, click `Save mapping` and run the cells afterward (ignore any error messages that may appear there beforehand). You can always come back here and redo your mappings if you change your mind.

In [8]:
%matplotlib widget
tiling_utils.tma_interactive_remap(
    manual_fov_regions,
    auto_fov_regions,
    slide_data,
    mapping_path,
    draw_radius=7,
    figsize=(7, 7),
    check_dist=check_dist,
    check_duplicates=check_duplicates,
    check_mismatches=check_mismatches
)

### Use proposed-to-auto mapping to rename FOVs in `manual_fov_regions`

In [ ]:
# load the mapping saved by the interactive visualization in
with open(mapping_path, 'r', encoding='utf-8') as mp:
    mapping = json.load(mp)

* `randomize`: shuffle the order of the FOVs in `remapped_fov_regions`
* `moly_insert`: insert a Moly point (specified by `moly_path`) between a specified interval of FOVs
* `moly_interval`: if `moly_insert` is set, insert Moly points at this interval

In [ ]:
randomize = True
moly_insert = True
moly_interval = 5

In [ ]:
# rename FOVs, randomize the order, and insert Moly points at a specified interval
remapped_fov_regions = tiling_utils.remap_and_reorder_fovs(
    manual_fov_regions,
    mapping,
    moly_path,
    randomize=randomize,
    moly_insert=moly_insert,
    moly_interval=moly_interval
)

In [ ]:
# save remapped_fov_regions
with open(remapped_fov_path, 'w', encoding='utf-8') as rtp:
    json.dump(remapped_fov_regions, rtp)